In [15]:
import ray

ray.init("ray://localhost:10001")

ClientContext(dashboard_url='127.0.0.1:8265', python_version='3.7.10', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', protocol_version='2022-03-16', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7facf2418350>)

In [17]:
# Before cluster resize
ray.cluster_resources()

{'node:172.31.0.194': 1.0,
 'memory': 17431498344.0,
 'object_store_memory': 7987059916.0,
 'node:172.31.13.216': 1.0,
 'CPU': 8.0}

In [44]:
%%sh

# I was hoping to do a manual resize here, but ala
REGION=$(curl --silent http://169.254.169.254/latest/dynamic/instance-identity/document | jq -r .region)
INSTANCE=`wget -q -O - http://169.254.169.254/latest/meta-data/instance-id`
CLUSTER_ID=$(aws --region ${REGION} ec2 describe-instances --instance-ids $INSTANCE --query 'Reservations[0].Instances[0].Tags[?Key==`aws:elasticmapreduce:job-flow-id`].Value' | jq -r '.[0]')
CORE_INSTANCE_GROUP=$(aws --region ${REGION} emr list-instances --cluster-id ${CLUSTER_ID} --instance-group-types "CORE" | jq -r '.Instances[0].InstanceGroupId')

echo "Resizing ${CORE_INSTANCE_GROUP} on ${CLUSTER_ID}"

aws --region ${REGION} emr modify-instance-groups --instance-groups InstanceGroupId=${CORE_INSTANCE_GROUP},InstanceCount=2


Resizing ig-1VN1F0JIC2H1P on j-FR2PRWRFQ5BZ



An error occurred (AccessDeniedException) when calling the ModifyInstanceGroups operation: User: arn:aws:sts::568026268536:assumed-role/EMR_EC2_DefaultRole/i-0a49a4c9676ee2ddb is not authorized to perform: elasticmapreduce:ModifyInstanceGroups on resource: * because no identity-based policy allows the elasticmapreduce:ModifyInstanceGroups action


CalledProcessError: Command 'b'\nREGION=$(curl --silent http://169.254.169.254/latest/dynamic/instance-identity/document | jq -r .region)\nINSTANCE=`wget -q -O - http://169.254.169.254/latest/meta-data/instance-id`\nCLUSTER_ID=$(aws --region ${REGION} ec2 describe-instances --instance-ids $INSTANCE --query \'Reservations[0].Instances[0].Tags[?Key==`aws:elasticmapreduce:job-flow-id`].Value\' | jq -r \'.[0]\')\nCORE_INSTANCE_GROUP=$(aws --region ${REGION} emr list-instances --cluster-id ${CLUSTER_ID} --instance-group-types "CORE" | jq -r \'.Instances[0].InstanceGroupId\')\n\necho "Resizing ${CORE_INSTANCE_GROUP} on ${CLUSTER_ID}"\n\naws --region ${REGION} emr modify-instance-groups --instance-groups InstanceGroupId=${CORE_INSTANCE_GROUP},InstanceCount=2\n'' returned non-zero exit status 255.

In [3]:
# After cluster resize
ray.cluster_resources()

{'memory': 28497285941.0,
 'node:172.31.9.40': 1.0,
 'CPU': 12.0,
 'object_store_memory': 12738014821.0,
 'node:172.31.10.50': 1.0,
 'node:172.31.9.84': 1.0}

In [12]:
ray.shutdown()

In [16]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import ray
ray.shutdown()

import joblib
from ray.util.joblib import register_ray

register_ray()
#ray.init(address="localhost:6379")
ray.init("ray://localhost:10001")

#import mlflow
#mlflow.set_tracking_uri('http://127.0.0.1:5001')
# mlflow.autolog() #log_models=False, exclusive=True)
import sklearn
#mlflow.sklearn.autolog()#log_models=True)

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=2, n_iter=5, verbose=10)

with joblib.parallel_backend('ray'):
    search.fit(digits.data, digits.target)

2022-06-29 22:59:16,120	WARNING pool.py:591 -- The 'context' argument is not supported using ray. Please refer to the documentation for how to control ray initialization.


Fitting 2 folds for each of 5 candidates, totalling 10 fits
(PoolActor pid=23237, ip=172.31.0.194) [CV 2/2; 1/5] START C=2.592943797404667e-06, class_weight=None, gamma=3.562247890262444e-08, tol=0.0002592943797404667
(PoolActor pid=23238, ip=172.31.0.194) [CV 1/2; 4/5] START C=1268.9610031679233, class_weight=None, gamma=0.0009236708571873865, tol=0.005736152510448681
(PoolActor pid=23237, ip=172.31.0.194) [CV 2/2; 1/5] END C=2.592943797404667e-06, class_weight=None, gamma=3.562247890262444e-08, tol=0.0002592943797404667;, score=0.352 total time=   0.2s
(PoolActor pid=23237, ip=172.31.0.194) [CV 2/2; 5/5] START C=8531.678524172814, class_weight=balanced, gamma=5.736152510448681e-06, tol=0.02395026619987486
(PoolActor pid=23238, ip=172.31.0.194) [CV 1/2; 4/5] END C=1268.9610031679233, class_weight=None, gamma=0.0009236708571873865, tol=0.005736152510448681;, score=0.959 total time=   0.1s
(PoolActor pid=10110) [CV 2/2; 2/5] START C=0.00011721022975334806, class_weight=balanced, gamma=0